This notebook contains the Cross-references implementation in Weaviate Classes

Appln 1: blog post class could have a cross-reference property called hasAuthor to link each post to its author,

Appln 2: chunk class could have a cross-reference property called sourceDocument to link each chunk to its original document.

Appln 3: A product (source) to its manufacturer (target).

Appln 4: A quiz item (source) to its category (target).

We will refer to the originating object as the **source** object, and the object that is being linked to (cross-referenced object) as the **target** object.

Example: A hasAuthor cross-reference property might be directed to the Author class, while a sourceDocument cross-reference property might be directed to the Document class.

Cross-references are uni-directional; to establish a bi-directional relationship, two distinct cross-reference properties are required, facilitating linkage in both directions.

In [ ]:
# class with cross reference to Author class
x = {
    "class": "BlogPost",
    "properties": [
        ...  // other class properties
        {
            "name": "hasAuthor",
            "dataType": ["Author"],
        },
    ],
    ...  // other class attributes (e.g. vectorizer)
}

### To create a cross-reference, Weaviate requires the following information:

- The class and UUID of the source (from) object.

- The class and UUID of the target (to) object.

- The name of the cross-reference property.


In [2]:
class Weaviate:
    def __init__(self, host_addr):
        self.client = host_addr 

    def __str__(self):
        return self.client

client = Weaviate(host_addr='http://localhost:8888')
print(client)

http://localhost:8888


In [ ]:
questions = client.collections.get("JeopardyQuestion")

questions.data.reference_add(
    from_uuid=question_obj_id,
    from_property="hasCategory",
    to=category_obj_id
)

Example:

where a document chunk includes a cross-reference to its original document, you can use the cross-reference to retrieve properties of the original document. Accordingly, you can retrieve the title of the document or the author of the document, just as you would retrieve a property of the chunk itself such as the text of the chunk.

In [ ]:
 import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_references=[
        wvc.query.QueryReference(
            link_on="hasCategory",
            return_properties=["title"]
        ),
    ],
    limit=2
)

for o in response.objects:
    print(o.properties["question"])
    # print referenced objects
    for ref_obj in o.references["hasCategory"].objects:
        print(ref_obj.properties)

In [ ]:
jeopardy = client.collections.get("JeopardyQuestion")
    response = jeopardy.query.fetch_objects(
        filters=wvc.query.Filter.by_ref(link_on="hasCategory").by_property("title").like("*Sport*"),
        return_references=wvc.query.QueryReference(link_on="hasCategory", return_properties=["title"]),
        limit=3
    )

    for o in response.objects:
        print(o.properties)
        print(o.references["hasCategory"].objects[0].properties["title"])

In [ ]:
class_definitions = [
    {
        "class": "JeopardyCategory",
        "properties": [
            {"name": "title", "dataType": ["text"]},
        ],
    },  # class 1 is created
    {
        "class": "JeopardyQuestion",
        "description": "A Jeopardy! question",
        "properties": [
            {"name": "question", "dataType": ["text"]},
            {"name": "answer", "dataType": ["text"]},
            {
                "name": "hasCategory",
                "dataType": ["JeopardyCategory"],
                "description": "The category of the question",
            },
        ],
    },  # class 2 is created with "datatype" as ["name of class"]
]

client.schema.create({"classes": class_definitions})

In [ ]:
client.schema.property.create("JeopardyCategory", {
    "name": "hasQuestion",
    "dataType": ["JeopardyQuestion"]
})

# Here the hasQuestion cross-reference property to JeopardyCategory

In [ ]:
sf_id = "00ff6900-e64f-5d94-90db-c8cfa3fc851b"
us_cities_id = "20ffc68d-986b-5e71-a680-228dba18d7ef"

client.data_object.reference.add(
    from_class_name="JeopardyQuestion",
    from_uuid=sf_id,
    from_property_name="hasCategory",
    to_class_name="JeopardyCategory",
    to_uuid=us_cities_id,
)

# cross referencing is happening in this cell

In [ ]:
category_definition = {
    "class": "JeopardyCategory",
    "properties": [
        {"name": "title", "dataType": ["text"]},
    ],
}

client.schema.create_class(category_definition)

# here the new class is created using above definition

In [ ]:
sf_id = "00ff6900-e64f-5d94-90db-c8cfa3fc851b"
us_cities_id = "20ffc68d-986b-5e71-a680-228dba18d7ef"

# For the "San Francisco" JeopardyQuestion object, add a cross-reference to the "U.S. CITIES" JeopardyCategory object
client.data_object.reference.add(
    from_class_name="JeopardyQuestion",
    from_uuid=sf_id,
    from_property_name="hasCategory",
    to_class_name="JeopardyCategory",
    to_uuid=us_cities_id,
)

# For the "U.S. CITIES" JeopardyCategory object, add a cross-reference to "San Francisco"
client.data_object.reference.add(
    from_class_name="JeopardyCategory",
    from_uuid=us_cities_id,
    from_property_name="hasQuestion",
    to_class_name="JeopardyQuestion",
    to_uuid=sf_id,
)